yolov3은 총 3개의 scale이 출력된다. 이때 미리 정의되어있는 3개의 다른 anchor box를 가지고 각각의 scale에서 객체를 탐지한다.<br>
3개의 scale x 서로다른 3개의 anchor box = 9 가 된다.<br>
즉, anchor box를 총 9번 사용한다.

In [56]:
import torch

import numpy as np
import pandas as pd

from sklearn.cluster import KMeans

In [2]:
df = pd.read_csv('./train/final.csv')

In [3]:
df.head()

,img_name,class,center_x,center_y,w,h
0,C:\Users\JungHyeona\Documents\deeplearning_sem...,0,0.5083,1.5664,0.2822,0.5352
1,C:\Users\JungHyeona\Documents\deeplearning_sem...,0,0.4541,1.4453,0.2891,0.4961
2,C:\Users\JungHyeona\Documents\deeplearning_sem...,0,0.4385,1.5928,0.3984,0.6348
3,C:\Users\JungHyeona\Documents\deeplearning_sem...,0,0.4697,1.2646,0.1641,0.3223
4,C:\Users\JungHyeona\Documents\deeplearning_sem...,0,0.5103,1.6006,0.3037,0.5527


anchors = [[(0.28, 0.22), (0.38, 0.48), (0.9, 0.78)]]

예시 값을 기준으로 볼 때 내가 탐지해야하는 객체는 총 4가지이다.<br>
때문에 나는 스케일 한개에 총 4개의 값이 있어햐 함.<br>
-> 밑에 함수를 수정해주어야 함.

1. if문으로 class별로 값을 구하기
2. 각 scale마다 값을 변경해서 구해야함.

In [92]:
# K-means 클러스터링을 사용하여 앵커 박스 크기를 결정
def kmeans_anchors(object_sizes, num_anchors, num_iterations=1000):
    num_objects = len(object_sizes)
    object_sizes = np.array(object_sizes)
    
    np.random.seed(42)
    # K-means++ 초기화
    centroids = [object_sizes[np.random.choice(num_objects)]]
    for _ in range(num_anchors - 1):
        distances = np.min(np.linalg.norm(object_sizes[:, None] - centroids, axis=2), axis=1)
        probabilities = distances / distances.sum()
        new_centroid = object_sizes[np.random.choice(num_objects, p=probabilities)]
        centroids.append(new_centroid)
    
    anchors = np.array(centroids)
    
    for _ in range(num_iterations):
        # 각 객체에 대해 가장 가까운 앵커 박스 인덱스 계산
        anchor_indices = np.argmin(np.linalg.norm(object_sizes[:, None] - anchors, axis=2), axis=1)

        # 새로운 앵커 박스 위치 계산
        new_anchors = np.array([object_sizes[anchor_indices == i].mean(axis=0) for i in range(num_anchors)])

        # 앵커 박스 위치 업데이트
        anchors = new_anchors

    return anchors

In [94]:
# 앵커 박스 크기를 5개로 설정하고자 한다면 다음과 같이 호출합니다.
num_anchors = 9

# 데이터셋에서 객체의 크기를 리스트로 저장
#object_widths = df['w'].tolist()
#object_heights = df['h'].tolist()
#object_size = list(zip(object_widths, object_heights))
object_sizes = np.array([(10, 13), (16, 30), (33, 23), (30, 61), (62, 45), (59, 119), (116, 90), (156, 198), (373, 326)])

anchors = kmeans_anchors(object_size, num_anchors)
print(anchors)

[[0.15058857 0.11373818]
 [0.22650095 0.40549976]
 [0.26754979 0.47092382]
 [0.36907619 0.62416429]
 [0.17728742 0.3335956 ]
 [0.207473   0.08130346]
 [0.31074103 0.5441398 ]
 [0.17144107 0.17693527]
 [0.15117643 0.06036135]]


In [ ]:
[[(0.15058857 0.11373818), (0.22650095 0.40549976), (0.26754979 0.47092382)],
 [(0.36907619 0.62416429), (0.17728742 0.33359560), (0.207473   0.08130346)],
 [(0.31074103 0.5441398), (0.17144107 0.1769352), (0.15117643 0.06036135)]]

In [ ]:
[[(0.1505, 0.1137), (0.2265, 0.4055), (0.2675, 0.4709)],
 [(0.3691, 0.6242), (0.1773, 0.3336), (0.2075, 0.0813)],
 [(0.3107, 0.5441), (0.1714, 0.1769), (0.1512, 0.0604)]]